<h1> Libraries </h1>

In [5]:
import os, sys
import numpy as np
import cv2
import scipy
import moviepy.editor as mpe
from moviepy.video.io.bindings import mplfig_to_npimage

from matplotlib import pyplot as plt

%matplotlib inline

np.set_printoptions(precision=4, linewidth=100)

In [6]:
# # Run following code if you occurred error in below cell
# import imageio
# imageio.plugins.ffmpeg.download()

In [19]:
path_003 = "../Handson_BR/dataset/Video_003/Video_003.avi"
path_001 = "./dataset/Video_008/Video_008.avi"
video = mpe.VideoFileClip(path_001)
video.subclip(0,50).ipython_display(width=500)

100%|█████████▉| 1250/1251 [00:00<00:00, 1744.46it/s]


In [20]:
print("Video duration: {}".format(video.duration))
print("Every frame size is {s[0]} * {s[1]}".format(s=video.size))

Video duration: 1318.68
Every frame size is 320 * 240


<h1> Create Matrix </h1>

In [21]:
def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.299, 0.587, 0.114])

In [22]:
def create_data_matrix_from_video(clip, fps=5, scale=50):
    return np.vstack([scipy.misc.imresize(rgb2gray(clip.get_frame(i / float(fps))).astype(int), scale).flatten() 
                      for i in range(fps * int(clip.duration))]).T

In [23]:
# Change resolution of image
scale = 75 # scale to X percent (100 means no scaling). CHANGE THIS FOR BETTER RESOLUTIONoriginal_width = video.size[1]
original_width = video.size[1]
original_height = video.size[0]
dims = (int(original_width * scale / 100), int(original_height * scale / 100))
print("Scale every frame to {d[0]} * {d[1]}".format(d=dims)) # single frame dimensions (height x width)

Scale every frame to 180 * 240


In [24]:
original_width

240

In [25]:
# Take times
fps = 100
M = create_data_matrix_from_video(video, fps, scale)

KeyboardInterrupt: 

### add testing frame

In [ ]:
# test_frame = np.vstack([scipy.misc.imresize(rgb2gray(cv2.imread("./dataset/Video_003/private_truth/color/color"+str(i)+".bmp")),scale).flatten() for i in range(1,16)]).T
test_frame = np.vstack([scipy.misc.imresize(rgb2gray(cv2.imread("./dataset/Video_001/private_truth/color/color"+str(i)+".bmp")),scale).flatten() for i in range(1,64)]).T

In [ ]:
# combine test_frame with 
M = np.hstack((M,test_frame))

In [ ]:
plt.imshow(np.reshape(M[:, 140], dims), cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(M, cmap='gray')
plt.show()

<h1> Background Removal </h1>

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
M = M.astype(float)
U, Sigma, Vt = svds(M, k=2)

In [ ]:
U.shape, Sigma.shape, Vt.shape

In [ ]:
low_rank = U @ np.diag(Sigma) @ Vt # low_rank is background

In [ ]:
plt.imshow(np.reshape(low_rank[:, 140], dims), cmap='gray')
plt.show()

In [ ]:
plt.imshow(np.reshape(low_rank[:, -5], dims), cmap='gray')
plt.show()

In [ ]:
plt.imshow(np.reshape(M[:, 140] - low_rank[:, 140], dims), cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank, cmap='gray')
plt.show()

In [ ]:
low_rank.shape

In [ ]:
people = M - low_rank

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(people, cmap='gray');

In [ ]:
plt.imshow(np.reshape(people[:, -15], dims), cmap='gray');

In [ ]:
# transform the grayscale image to black and white for validation
(thresh, im_bw) = cv2.threshold(np.reshape(people[:, -15],dims), -30, 255, cv2.THRESH_BINARY_INV)
plt.imshow(im_bw, cmap='gray')

###  Validation

In [ ]:
def dice_difference(prediction, groundtruth, k=255):
    dice = np.sum(prediction[groundtruth==k])*2.0 / (np.sum(prediction) + np.sum(groundtruth))
    return dice

In [ ]:
predict_frame = []
for i in range(-15,0):
    _, im_bw = cv2.threshold(np.reshape(people[:, i],dims), -30, 255, cv2.THRESH_BINARY_INV)
    predict_frame.append(im_bw)
predict_frame = np.array(predict_frame)
valid_frame = np.array([scipy.misc.imresize(
    cv2.imread("./dataset/Video_003/private_truth/"+str(i)+".bmp")[:,:,0],scale) 
                        for i in range(1,16)])

In [ ]:
len(predict_frame)

In [ ]:
average_dice_difference = np.mean([dice_difference(predict_frame[p], valid_frame[p]) for p in range(15)])
average_dice_difference

<h1> Make Video </h1>

In [ ]:
people_frames = people.reshape(dims + (-1,))
people_frames.shape

In [ ]:
# this takes a while
fig, ax = plt.subplots()
def make_frame(t):
    ax.clear()
    ax.imshow(people_frames[..., int(t*fps)], cmap="gray")
    return mplfig_to_npimage(fig)

animation = mpe.VideoClip(make_frame, duration=int((video.duration-1)*0.1))
animation.write_videofile('./car_%s.mp4'%str(scale), fps=fps)

In [ ]:
video = mpe.VideoFileClip("./car_%s.mp4"%str(scale))
video.ipython_display(width=500)

In [ ]:
im_bw.shape

### Reference
- https://zulko.github.io/moviepy/getting_started/working_with_matplotlib.html